In [1]:
import argparse
import os
path = os.getcwd()
path=path[:path.find('timeseries-WGAN')+15]
# No = (os.path.basename(__file__))[-4]
No = "0_3" # notebook用
print('実験No.'+No)
import warnings
warnings.simplefilter('ignore')# 警告を非表示
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import math
import sys
sys.path.append(path+"/")
import random
import time
import statsmodels.api as sm
from scipy.stats import norm
import japanize_matplotlib
from scipy.stats import gaussian_kde

import torchvision.transforms as transforms
from torchvision.utils import save_image

import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.autograd import Variable
# 人工データを生成してくれる機械が置いてあるところ
import tsModel
# 学習用のニューラルネットが置いてあるところ
import models
# p-Wasserstein距離の関数
import Wasserstein

実験No.0_3


In [2]:
# "output-images"フォルダを作成（既にあるならそれで良し）
os.makedirs("output-images", exist_ok=True)
os.makedirs("parameters", exist_ok=True)

In [3]:
# 真のモデルのパラメータ
phi_ast=[0.3,-0.4,0.2,-0.5,0.6,-0.1,0.1]
p_ast=len(phi_ast)
mu_ast=0
sigma_ast=2

In [4]:
# データセットの作成
trainT=1000
n=100
data_index = range(n)
trainDataSets=[]
for seed in data_index:
    trainData = tsModel.SARIMA(a=phi_ast, N=trainT, random_seed=seed, mu=mu_ast, sigma=sigma_ast)
    trainDataSets.append(trainData)

In [5]:
# 学習する推定モデルの形状や学習方法なんかを決定します
# 学習時のハイパラの決定（入力を受け付ける）
parser = argparse.ArgumentParser()
# ランダムシードについて
parser.add_argument("--generator_seed", type=int, default=0, help="generatorのパラメータの初期値のシード")
parser.add_argument("--discriminator_seed", type=int, default=0, help="discriminatorのパラメータの初期値のシード")
parser.add_argument("--predictor_seed", type=int, default=0, help="predictorのパラメータの初期値のシード")
parser.add_argument("--training_seed", type=int, default=0, help="訓練データを学習させる順番を決めるシード")
parser.add_argument("--data_seed", type=int, default=0, help="Dataの作成に用いる乱数のseed")
# 学習方法について
parser.add_argument("--n_epochs", type=int, default=2000, help="Discriminatorを学習させる回数")
parser.add_argument("--batch_size", type=int, default=64, help="batchの大きさ")
parser.add_argument("--lr", type=float, default=0.00005, help="学習率")
parser.add_argument("--n_critic", type=int, default=5, help="一度generatorを更新するごとに何回discriminatorを更新するか")
parser.add_argument("--discriminator_hidden_unit", type=int, default=64, help="discriminatorの隠れ層のニューロンの数")
# parser.add_argument("--withGP", type=bool, default=True, help="clipingの代わりにGradient Penaltyを加えるかどうか。True/False")
# parser.add_argument("--withCorr", type=bool, default=True, help="Generatorの出力がbatch方向に無相関になるようなロスを加えるかどうか。　True/False")
# モデルの保存やLossの可視化について
parser.add_argument("--sample_interval", type=int, default=100, help="epochを何回まわす度にモデルの保存を行うか")

try:
    opt = parser.parse_args() # .pyの場合はこちらを使用(.ipynbの場合エラーになります)
except:
    opt = parser.parse_args(args=[]) # .ipynbの場合はこちらを使用

print(opt)

Namespace(batch_size=64, data_seed=0, discriminator_hidden_unit=64, discriminator_seed=0, generator_seed=0, lr=5e-05, n_critic=5, n_epochs=2000, predictor_seed=0, sample_interval=100, training_seed=0)


usage: ipykernel_launcher.py [-h] [--generator_seed GENERATOR_SEED]
                             [--discriminator_seed DISCRIMINATOR_SEED]
                             [--predictor_seed PREDICTOR_SEED]
                             [--training_seed TRAINING_SEED]
                             [--data_seed DATA_SEED] [--n_epochs N_EPOCHS]
                             [--batch_size BATCH_SIZE] [--lr LR]
                             [--n_critic N_CRITIC]
                             [--discriminator_hidden_unit DISCRIMINATOR_HIDDEN_UNIT]
                             [--sample_interval SAMPLE_INTERVAL]
ipykernel_launcher.py: error: unrecognized arguments: -f /run/user/1165/jupyter/kernel-d26b27d5-3c8d-483e-92d0-0681754f7350.json


In [6]:
# gpuが使えるかどうか
cuda = True if torch.cuda.is_available() else False
if cuda:
    print("GPUが使えます。")
    use_gpu = input('GPUを使いますか？ （Yes：1, No：0）  ----> ')
    cuda = bool(int(use_gpu))
else:
    print("GPUは使えません。")
if cuda:
    gpu_id = input('使用するGPUの番号を入れてください : ')
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu_id
device = torch.device('cuda:'+gpu_id if cuda else 'cpu')

GPUが使えます。
GPUを使いますか？ （Yes：1, No：0）  ----> 1
使用するGPUの番号を入れてください : 0


In [7]:
# 推定モデルの決定
p=7

In [8]:
os.makedirs("output-images/p{0}".format(p), exist_ok=True)
os.makedirs("parameters/p{0}".format(p), exist_ok=True)

In [9]:
torch.manual_seed(opt.generator_seed)
generator = models.LinearGenerator(p = p, input_dim=1, is_bias=False)
torch.manual_seed(opt.predictor_seed)
predictor = models.LinearPredictNet(p=p, input_dim=1, is_bias=True)

In [10]:
# 訓練データを一つ用いて学習させる
dataSeed=opt.data_seed
# こいつをtrain:validation=900:100に分割する
Data = trainDataSets[dataSeed]
Data = torch.tensor(Data, dtype=torch.float)
Data=Data.view(1,-1)
trainData = Data[:,:900]
valData = Data[:,900:]

In [11]:
# trainDataとvalDataを {𝑋𝑡}𝑡0𝑡=𝑡0−𝑝 ごとに取り出しやすいようにMatrixに変換する
trainMatrix = []
for i in range(trainData.shape[1]-(p)):
    ans = trainData[:,i:i+p+1].view(1,Data.shape[0],-1)
    trainMatrix.append(ans)
trainMatrix = torch.cat(trainMatrix)
valMatrix = []
for i in range(valData.shape[1]-(p)):
    ans = valData[:,i:i+p+1].view(1,Data.shape[0],-1)
    valMatrix.append(ans)
valMatrix = torch.cat(valMatrix)

In [12]:
# Optimizers(パラメータに対して定義される)
optimizer_G = torch.optim.RMSprop(params=generator.parameters(), lr=opt.lr)
# optimizer_D = torch.optim.RMSprop(params=discriminator.parameters(), lr=opt.lr)
optimizer_F = torch.optim.RMSprop(params=predictor.parameters(), lr=opt.lr)
# optimizer_F = torch.optim.Adam(params=predictor.parameters())

# 二条誤差MSE
mseLoss = nn.MSELoss()

# パラメータと学習データをGPUに乗っける
generator.to(device)
# discriminator.to(device)
predictor.to(device)
trainMatrix=trainMatrix.to(device)
valMatrix=valMatrix.to(device)
mseLoss.to(device)

saveModel = input('作成したモデルを {0} epochごとに逐次保存しますか？ （Yes：1, No：0）  ----> '.format(opt.sample_interval))
saveModel = bool(int(saveModel))

作成したモデルを 100 epochごとに逐次保存しますか？ （Yes：1, No：0）  ----> 1


In [14]:
# 訓練データの時系列のどの時刻を学習に用いるかをランダムにしているが、そのランダムシードを固定する
random.seed(a=opt.training_seed)

do_preTrain = bool(int(input('事前学習をここで行いますか、それとも読み込みますか （行う：1, 読み込む：0）  ----> ')))
pretrain_param = 'parameters/p{0}/No{1}_predictor_epoch{2}_batchSize{3}_DataSeed{4}.pth'.format(p, No, 0, opt.batch_size, dataSeed )
if not do_preTrain:
    try:# モデルパラメータを読み込もうとして失敗したら、それはファイルがないと言うことなので、事前学習をこの場で行う
        predictor.load_state_dict(torch.load(pretrain_param)) 
    except:
        print("モデルが存在しないので事前学習を行います")
        do_preTrain=True

if do_preTrain:
    # ここでまずはFの事前学習を行う
    loss_pre = []
    val_loss_pre = []
    pretrain_epoch = 1000
    start=time.time()
    for epoch in range(1, pretrain_epoch+1):# epochごとの処理
        # batchの処理は、0~892をランダムに並び替えたリストbatch_sampleを作成し、ここからbatch×(p+1)の学習データを一つづつ獲得する
        l=list(range(trainMatrix.shape[0]-opt.batch_size))
        batch_sample = random.sample(l, len(l))
        for i, batch in enumerate(batch_sample):
            X = trainMatrix[batch : batch+opt.batch_size]# torch.Size([64, 1, 8]) (batch, dim, p+1)
            optimizer_F.zero_grad()
            input_tensor = X[:,:,:-1]
            input_tensor = torch.cat([torch.randn([opt.batch_size,1,1]).to(device), input_tensor], dim=2)
            true_tensor = X[:,:,-1]
            prediction = predictor(input_tensor)
            loss_F = mseLoss(prediction, true_tensor)
            loss_F.backward()
            optimizer_F.step()
        loss_pre.append(loss_F.item())
        
        val_input = torch.cat([torch.randn([valMatrix.shape[0],1,1]).to(device), valMatrix[:,:,:-1]], dim=2)
        val_target = valMatrix[:,:,-1].view(valMatrix.shape[0], -1)
        val_loss = mseLoss(predictor(val_input), val_target)
        val_loss_pre.append(val_loss.item())
        
        print("epoch：{0}/{1}   loss_F：{2: .4f}   経過時間：{3: .1f}秒".format(epoch, pretrain_epoch, round(loss_F.item(), 4), time.time()-start))
        if epoch % 100==0:
            plt.figure(figsize=(13,8))
            plt.title("PredictorのLossの遷移　\n　batchSize:{0}".format(opt.batch_size))
            plt.xlabel("epoch")
            plt.ylabel("Loss")
            plt.plot(loss_pre, label="training")
            plt.plot(val_loss_pre, label="validation")
            plt.legend()
            plt.savefig("preloss.png")
            plt.close()
    torch.save(predictor.state_dict(), pretrain_param)
    
    plt.figure(figsize=(13,8))
    plt.title("PredictorのLossの遷移　\n　batchSize:{0}".format(opt.batch_size))
    plt.xlabel("epoch")
    plt.ylabel("Loss")
    plt.plot(loss_pre, label="training")
    plt.plot(val_loss_pre, label="validation")
    plt.legend()
    plt.savefig("output-images/p{0}/No{1}_preloss_epoch{2}_batchSize{3}_DataSeed{4}.png".format(p, No, epoch, opt.batch_size, dataSeed ))
    plt.close()
    print("pre-training終了")


事前学習をここで行いますか、それとも読み込みますか （行う：1, 読み込む：0）  ----> 1
epoch：1/1000   loss_F： 8.6638   経過時間： 1.1秒
epoch：2/1000   loss_F： 7.3150   経過時間： 2.2秒
epoch：3/1000   loss_F： 8.0234   経過時間： 3.4秒
epoch：4/1000   loss_F： 6.7098   経過時間： 4.5秒
epoch：5/1000   loss_F： 3.6953   経過時間： 5.6秒
epoch：6/1000   loss_F： 6.9309   経過時間： 6.7秒
epoch：7/1000   loss_F： 3.6304   経過時間： 7.9秒
epoch：8/1000   loss_F： 3.5588   経過時間： 9.0秒
epoch：9/1000   loss_F： 6.3740   経過時間： 10.2秒
epoch：10/1000   loss_F： 2.8806   経過時間： 11.3秒
epoch：11/1000   loss_F： 4.6801   経過時間： 12.5秒
epoch：12/1000   loss_F： 4.0022   経過時間： 13.6秒
epoch：13/1000   loss_F： 5.2313   経過時間： 14.8秒
epoch：14/1000   loss_F： 5.5657   経過時間： 15.9秒
epoch：15/1000   loss_F： 2.6174   経過時間： 17.1秒
epoch：16/1000   loss_F： 3.4789   経過時間： 18.2秒
epoch：17/1000   loss_F： 4.7815   経過時間： 19.4秒
epoch：18/1000   loss_F： 3.0884   経過時間： 20.5秒
epoch：19/1000   loss_F： 3.4341   経過時間： 21.7秒
epoch：20/1000   loss_F： 3.7218   経過時間： 22.8秒
epoch：21/1000   loss_F： 3.2312   経過時間： 24.0秒
epoch：22/1000   loss_F

epoch：179/1000   loss_F： 2.5746   経過時間： 188.7秒
epoch：180/1000   loss_F： 3.2849   経過時間： 189.6秒
epoch：181/1000   loss_F： 3.0400   経過時間： 190.5秒
epoch：182/1000   loss_F： 4.6648   経過時間： 191.4秒
epoch：183/1000   loss_F： 4.1499   経過時間： 192.3秒
epoch：184/1000   loss_F： 4.6207   経過時間： 193.2秒
epoch：185/1000   loss_F： 3.7982   経過時間： 194.1秒
epoch：186/1000   loss_F： 4.6226   経過時間： 195.0秒
epoch：187/1000   loss_F： 3.1220   経過時間： 195.9秒
epoch：188/1000   loss_F： 5.0053   経過時間： 196.8秒
epoch：189/1000   loss_F： 3.6902   経過時間： 197.7秒
epoch：190/1000   loss_F： 3.5473   経過時間： 198.6秒
epoch：191/1000   loss_F： 3.8804   経過時間： 199.5秒
epoch：192/1000   loss_F： 3.5317   経過時間： 200.5秒
epoch：193/1000   loss_F： 3.4908   経過時間： 201.3秒
epoch：194/1000   loss_F： 2.8048   経過時間： 202.3秒
epoch：195/1000   loss_F： 3.3229   経過時間： 203.2秒
epoch：196/1000   loss_F： 3.3187   経過時間： 204.1秒
epoch：197/1000   loss_F： 4.6331   経過時間： 205.0秒
epoch：198/1000   loss_F： 3.4321   経過時間： 205.9秒
epoch：199/1000   loss_F： 3.6622   経過時間： 206.8秒
epoch：200/100

epoch：354/1000   loss_F： 3.4551   経過時間： 370.7秒
epoch：355/1000   loss_F： 3.5757   経過時間： 371.8秒
epoch：356/1000   loss_F： 4.4185   経過時間： 373.0秒
epoch：357/1000   loss_F： 3.2049   経過時間： 374.1秒
epoch：358/1000   loss_F： 2.7748   経過時間： 375.3秒
epoch：359/1000   loss_F： 3.4735   経過時間： 376.4秒
epoch：360/1000   loss_F： 3.2457   経過時間： 377.5秒
epoch：361/1000   loss_F： 2.8268   経過時間： 378.7秒
epoch：362/1000   loss_F： 3.5553   経過時間： 379.8秒
epoch：363/1000   loss_F： 3.2778   経過時間： 381.0秒
epoch：364/1000   loss_F： 3.7864   経過時間： 382.1秒
epoch：365/1000   loss_F： 3.5270   経過時間： 383.2秒
epoch：366/1000   loss_F： 5.1680   経過時間： 384.4秒
epoch：367/1000   loss_F： 3.9945   経過時間： 385.5秒
epoch：368/1000   loss_F： 2.8121   経過時間： 386.6秒
epoch：369/1000   loss_F： 4.4173   経過時間： 387.8秒
epoch：370/1000   loss_F： 2.6474   経過時間： 388.9秒
epoch：371/1000   loss_F： 3.3130   経過時間： 390.1秒
epoch：372/1000   loss_F： 2.9613   経過時間： 391.2秒
epoch：373/1000   loss_F： 4.4907   経過時間： 392.4秒
epoch：374/1000   loss_F： 3.0031   経過時間： 393.4秒
epoch：375/100

epoch：529/1000   loss_F： 3.5974   経過時間： 561.3秒
epoch：530/1000   loss_F： 3.9112   経過時間： 562.4秒
epoch：531/1000   loss_F： 4.1060   経過時間： 563.5秒
epoch：532/1000   loss_F： 3.6589   経過時間： 564.6秒
epoch：533/1000   loss_F： 3.8077   経過時間： 565.8秒
epoch：534/1000   loss_F： 3.7845   経過時間： 567.0秒
epoch：535/1000   loss_F： 4.1156   経過時間： 568.1秒
epoch：536/1000   loss_F： 4.6494   経過時間： 569.3秒
epoch：537/1000   loss_F： 3.7018   経過時間： 570.4秒
epoch：538/1000   loss_F： 3.9222   経過時間： 571.6秒
epoch：539/1000   loss_F： 3.3364   経過時間： 572.7秒
epoch：540/1000   loss_F： 3.3631   経過時間： 573.8秒
epoch：541/1000   loss_F： 3.7805   経過時間： 575.0秒
epoch：542/1000   loss_F： 2.7236   経過時間： 576.2秒
epoch：543/1000   loss_F： 2.9026   経過時間： 577.4秒
epoch：544/1000   loss_F： 3.3742   経過時間： 578.5秒
epoch：545/1000   loss_F： 5.1770   経過時間： 579.7秒
epoch：546/1000   loss_F： 3.4835   経過時間： 580.9秒
epoch：547/1000   loss_F： 4.3524   経過時間： 582.0秒
epoch：548/1000   loss_F： 3.4960   経過時間： 583.1秒
epoch：549/1000   loss_F： 3.1992   経過時間： 584.2秒
epoch：550/100

epoch：704/1000   loss_F： 3.7829   経過時間： 761.0秒
epoch：705/1000   loss_F： 2.9306   経過時間： 762.1秒
epoch：706/1000   loss_F： 4.6236   経過時間： 763.3秒
epoch：707/1000   loss_F： 4.3625   経過時間： 764.5秒
epoch：708/1000   loss_F： 3.6153   経過時間： 765.6秒
epoch：709/1000   loss_F： 3.7305   経過時間： 766.8秒
epoch：710/1000   loss_F： 3.0371   経過時間： 767.9秒
epoch：711/1000   loss_F： 2.8272   経過時間： 769.1秒
epoch：712/1000   loss_F： 3.1748   経過時間： 770.2秒
epoch：713/1000   loss_F： 3.5319   経過時間： 771.4秒
epoch：714/1000   loss_F： 3.2715   経過時間： 772.6秒
epoch：715/1000   loss_F： 3.3595   経過時間： 773.7秒
epoch：716/1000   loss_F： 2.8664   経過時間： 774.9秒
epoch：717/1000   loss_F： 3.4333   経過時間： 776.0秒
epoch：718/1000   loss_F： 3.7329   経過時間： 777.2秒
epoch：719/1000   loss_F： 5.4009   経過時間： 778.2秒
epoch：720/1000   loss_F： 3.0639   経過時間： 779.3秒
epoch：721/1000   loss_F： 2.9158   経過時間： 780.4秒
epoch：722/1000   loss_F： 3.3964   経過時間： 781.6秒
epoch：723/1000   loss_F： 3.2458   経過時間： 782.7秒
epoch：724/1000   loss_F： 3.2905   経過時間： 783.9秒
epoch：725/100

epoch：879/1000   loss_F： 3.9094   経過時間： 961.3秒
epoch：880/1000   loss_F： 4.1132   経過時間： 962.4秒
epoch：881/1000   loss_F： 3.9291   経過時間： 963.6秒
epoch：882/1000   loss_F： 4.2701   経過時間： 964.8秒
epoch：883/1000   loss_F： 3.8592   経過時間： 965.9秒
epoch：884/1000   loss_F： 3.4064   経過時間： 967.1秒
epoch：885/1000   loss_F： 3.6613   経過時間： 968.3秒
epoch：886/1000   loss_F： 3.7822   経過時間： 969.4秒
epoch：887/1000   loss_F： 2.5661   経過時間： 970.5秒
epoch：888/1000   loss_F： 5.2063   経過時間： 971.7秒
epoch：889/1000   loss_F： 3.3703   経過時間： 972.9秒
epoch：890/1000   loss_F： 3.4434   経過時間： 974.0秒
epoch：891/1000   loss_F： 2.9203   経過時間： 975.2秒
epoch：892/1000   loss_F： 3.1713   経過時間： 976.4秒
epoch：893/1000   loss_F： 4.4006   経過時間： 977.5秒
epoch：894/1000   loss_F： 4.3755   経過時間： 978.6秒
epoch：895/1000   loss_F： 2.8277   経過時間： 979.8秒
epoch：896/1000   loss_F： 3.7959   経過時間： 981.0秒
epoch：897/1000   loss_F： 4.6258   経過時間： 982.1秒
epoch：898/1000   loss_F： 4.6484   経過時間： 983.1秒
epoch：899/1000   loss_F： 3.0623   経過時間： 984.2秒
epoch：900/100

In [15]:
# hat_sigmaに相当する部分がほとんどになってるので1にする
predictor.fc1.weight.data[0][0] = torch.tensor(1)

In [18]:
hat_mu = float(predictor.state_dict()['fc1.bias'])
print(mu_ast)
print(hat_mu)
hat_sigma = float(predictor.state_dict()['fc1.weight'][0][0])
print(sigma_ast)
print(hat_sigma)
hat_phi = [round(float(predictor.state_dict()['fc1.weight'][0][p-i]),3) for i in range(0, p)]
print(phi_ast)
print(hat_phi)
print(sum([np.abs(phi_ast[i]-hat_phi[i]) for i in range(len(hat_phi))])/len(hat_phi)) # 真の値との絶対誤差の平均

0
0.05302155762910843
2
1.0
[0.3, -0.4, 0.2, -0.5, 0.6, -0.1, 0.1]
[0.259, -0.381, 0.219, -0.523, 0.56, -0.056, 0.127]
0.03042857142857142


In [19]:
min_floss=np.inf# epochのflossのの最小値を保管
start=time.time()

batches_done = 0
epoch_done = 0# generatorを学習した回数
loss_curve = []

In [ ]:
for epoch in range(1, opt.n_epochs+1):# epochごとの処理(discriminatorのepoch)
    
    # epochごとにbatchで計算したlossを平均した値をloss_curveとして描きたい
    loss_list = []
    
    # batchの処理は、0~892をランダムに並び替えたリストbatch_sampleを作成し、ここからbatch×(p+1)の学習データを一つづつ獲得する
    l=list(range(trainMatrix.shape[0]-opt.batch_size))
    batch_sample = random.sample(l, len(l))
    for i, batch in enumerate(batch_sample):
        
        X = trainMatrix[batch : batch+opt.batch_size]# torch.Size([64, 1, 8]) (batch, dim, p+1)

        # generatorの勾配情報を0に初期化
        optimizer_F.zero_grad()
        optimizer_G.zero_grad()

        # 正規化されたinnoationの推定量をgeneratorを用いて算出
        hat_normeps_t = generator(X)
        # これと過去p時刻の時系列の値（X_{t-1}, .... , X_{t-p}）をpredictorへ入力
        input_tensor = torch.cat([hat_normeps_t.view(opt.batch_size, -1, 1), X[:,:,:-1]], dim=2)
        prediction = predictor(input_tensor)

        loss_G = Variable(Wasserstein.pWasserstein(hat_normeps_t.view(opt.batch_size), p=1), requires_grad=True).to(device)
        loss_F = mseLoss(prediction, X[:,:,-1])

        loss = loss_G+loss_F
        loss_list.append(loss.item())

        # lossを目的関数としてネットワークの全パラメータで微分をしてくれと言う合図
        loss.backward()
        # generatorのパラメータをその微分値とoptimizerを使って更新してくれ！
        optimizer_G.step()
        optimizer_F.step()

        generator_done+=1


    print("epoch：{0}/{1}   batch：{2:003}/{3}   loss_G：{4: .4f}   loss_F：{5: .4f}   経過時間：{6: .1f}秒".format(epoch, opt.n_epochs, i+1, len(batch_sample), round(float(loss_G), 4), round(float(loss_F), 4), time.time()-start))
            
    if saveModel and epoch % opt.sample_interval == 0:
        torch.save(generator.state_dict(), 'parameters/p'+str(p)+'/No{0}_generator_epoch{1}_batchSize{2}_DataSeed{3}.pth'.format(No, epoch, opt.batch_size, dataSeed))
        torch.save(predictor.state_dict(), 'parameters/p'+str(p)+'/No{0}_predictor_epoch{1}_batchSize{2}_DataSeed{3}.pth'.format(No, epoch, opt.batch_size, dataSeed))

    # epochごとにbatchで計算したlossを平均した値をloss_curveとして描きたい
#     try:
#         loss_D_curve.append(sum(loss_D_list)/len(loss_D_list))
#     except:
#         loss_D_curve.append(None)
    try:
        loss_G_curve.append(sum(loss_G_list)/len(loss_G_list))
    except:
        loss_G_curve.append(None)
    try:
        loss_F_curve.append(sum(loss_F_list)/len(loss_F_list))
    except:
        loss_F_curve.append(None)
    

    
    # validationデータによるlossも計算したい
    val_hat_normeps_t = generator(valMatrix)
    val_normeps_t = torch.randn_like(val_hat_normeps_t)
    val_input_tensor = torch.cat([val_hat_normeps_t.view(-1, 1,1), valMatrix[:,:,:-1]], dim=2)
    
#     val_loss_D = -torch.mean(discriminator(val_normeps_t)) + torch.mean(discriminator(val_hat_normeps_t))
#     if opt.withGP:
#         val_loss_D = val_loss_D + gradient_penalty(generated_data=val_hat_normeps_t, real_data=val_normeps_t, gp_weight=gp_weight) 
#     val_loss_D_curve.append(float(val_loss_D))
    val_loss_G = Wasserstein.pWasserstein(val_hat_normeps_t.view(-1), p=1)
    # if opt.withCorr:
    #     val_loss_G = val_loss_G + corr_weight*corr(val_hat_normeps_t)
    val_loss_G_curve.append(float(val_loss_G))
    
    val_loss_F = mseLoss(predictor(val_input_tensor), valMatrix[:,:,0])
    val_loss_F_curve.append(float(val_loss_F))

    # val_loss_Fの最小値を保管
    if min_floss > val_loss_F_curve[-1]:
        min_floss=val_loss_F_curve[-1]
        torch.save(generator.state_dict(), 'parameters/p'+str(p)+'/No{0}_generator_minLoss_batchSize{2}_DataSeed{3}.pth'.format(No, epoch, opt.batch_size, dataSeed))
        torch.save(predictor.state_dict(), 'parameters/p'+str(p)+'/No{0}_predictor_minLoss_batchSize{2}_DataSeed{3}.pth'.format(No, epoch, opt.batch_size, dataSeed))

        print("validationのflossの最小値を更新しました　　Loss:", min_floss)
    
    if epoch % 10==0:
#         plt.figure(figsize=(13,8))
#         plt.title("DiscriminatorのLossの遷移　\n　batchSize:{0}, GPの係数:{1}, Corrの係数:{2}".format(opt.batch_size, gp_weight, corr_weight))
#         plt.xlabel("epoch")
#         plt.ylabel("Loss")
#         plt.plot(loss_D_curve, label="training")
#         plt.plot(val_loss_D_curve, label="validation")
#         plt.legend()
#         plt.savefig("dloss.png")
#         plt.close()

        plt.figure(figsize=(13,8))
        plt.title("GeneratorのLossの遷移　\n　batchSize:{0}".format(opt.batch_size))
        plt.xlabel("epoch")
        plt.ylabel("Loss")
        plt.plot(loss_G_curve, label="training")
        plt.plot(val_loss_G_curve, label="validation")
        plt.legend()
        plt.savefig("gloss.png")
        plt.close()
        
        plt.figure(figsize=(13,8))
        plt.title("PredictorのLossの遷移　\n　batchSize:{0}".format(opt.batch_size))
        plt.xlabel("epoch")
        plt.ylabel("Loss")
        plt.plot(loss_F_curve, label="training")
        plt.plot(val_loss_F_curve, label="validation")
        plt.legend()
        plt.savefig("floss.png")
        plt.close()
    
torch.save(generator.state_dict(), 'parameters/p'+str(p)+'/No{0}_generator_epoch{1}_batchSize{2}_DataSeed{3}.pth'.format(No, epoch, opt.batch_size, dataSeed))
torch.save(predictor.state_dict(), 'parameters/p'+str(p)+'/No{0}_predictor_epoch{1}_batchSize{2}_DataSeed{3}.pth'.format(No, epoch, opt.batch_size, dataSeed))
